In [14]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectFromModel

In [15]:
# Đọc dữ liệu từ các file CSV
message_gold_label = pd.read_csv('data/message_gold_label.csv')
potential = pd.read_csv('data/potential.csv')
campaign = pd.read_csv('data/campaigns.csv')

# Kết hợp các DataFrame thành một DataFrame duy nhất
data = pd.concat([message_gold_label, potential], ignore_index=True)
data = data[data['message_type'] == 'bulk']  # Chỉ giữ lại các dòng có message_type là 'bulk'

# Loại bỏ các cột không cần thiết
columns_to_drop = ['is_unsubscribed', 'unsubscribed_at', 'complained_at','is_complained', 'is_blocked', 
                   'blocked_at', 'purchased_at', 'date', 'campaign_message','opened_first_time_at',
                   'clicked_first_time_at','is_opened','is_clicked','client_id','message_id', 'is_purchased']
data = data.drop(columns=columns_to_drop)

columns_to_drop_campaign = ['is_test','position','hour_limit','warmup_mode','ab_test','started_at','finished_at','total_count']
campaign = campaign.drop(columns=columns_to_drop_campaign)
data

,campaign_id,message_type,channel,sent_at,opened_last_time_at,clicked_last_time_at,duration
4000,136,bulk,email,2021-05-19 07:50:47,2021-05-20 02:41:19,2021-05-20 02:41:19,22.0
4001,136,bulk,email,2021-05-19 07:07:20,2021-05-19 20:05:03,2021-05-19 20:05:03,31451.0
4002,136,bulk,email,2021-05-19 07:01:43,2021-05-20 11:45:38,2021-05-20 11:45:38,37.0
4003,136,bulk,email,2021-05-19 07:43:20,2021-05-20 11:38:03,2021-05-20 11:38:03,12.0
4004,136,bulk,email,2021-05-19 07:51:26,2021-05-19 16:06:28,2021-05-19 16:06:28,10.0
...,...,...,...,...,...,...,...
228880,361,bulk,email,2021-06-11 08:10:09,2021-06-11 08:17:33,2021-06-11 08:17:33,44.0
228881,361,bulk,email,2021-06-11 08:04:45,2021-06-11 09:30:11,2021-06-11 09:30:11,1639.0
228882,361,bulk,email,2021-06-11 08:05:53,2021-06-11 08:27:04,2021-06-11 08:27:04,7.0
228883,361,bulk,email,2021-06-11 08:08:00,2021-06-11 12:42:17,2021-06-11 12:42:17,9.0


In [16]:
# Kết hợp dữ liệu từ hai bảng data và campaign
merged_data = pd.merge(data, campaign, 
                       left_on=['campaign_id','message_type'], 
                       right_on=['id', 'campaign_type'])
merged_data = merged_data.drop(columns=['id','campaign_type','channel_y','campaign_id','message_type'])

In [17]:
# Hàm để trích xuất các đặc trưng thời gian từ các cột datetime
def extract_datetime_features(X, datetime_columns):
    for col in datetime_columns:
        if col in X.columns:
            X[col + '_weekday'] = pd.to_datetime(X[col]).dt.weekday  # Trích xuất ngày trong tuần
            X[col + '_hour'] = pd.to_datetime(X[col]).dt.hour  # Trích xuất giờ trong ngày
            X[col + '_month'] = pd.to_datetime(X[col]).dt.month  # Trích xuất tháng
            X = X.drop(columns=[col])  # Loại bỏ cột datetime gốc
    return X

merged_data_date_time_feature = extract_datetime_features(merged_data, ['sent_at','opened_last_time_at','clicked_last_time_at'])

In [18]:
# Mã hóa các cột phân loại và xử lý boolean
from sklearn.preprocessing import LabelEncoder

def drop_and_encode(df, encode_columns):
    le = LabelEncoder()
    for col in encode_columns:
        if col in df.columns:
            df[col] = df[col].fillna('missing').astype(str)  # Thay thế NaN bằng 'missing' để tránh lỗi
            df[col + '_encoded'] = le.fit_transform(df[col])  # Mã hóa cột thành số
            df = df.drop(columns=[col])  # Loại bỏ cột gốc sau khi mã hóa
    return df

def process_bool_columns(df, bool_columns):
    for col in bool_columns:
        if col in df.columns:
            df[col] = df[col].astype(int)  # Chuyển đổi boolean thành 0 và 1
    return df

def convert_to_numeric(df, numerical_columns):
    for col in numerical_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # Chuyển thành số và xử lý lỗi
    return df

encode_columns = ['topic', 'channel_x']
merged_data_date_time_feature = drop_and_encode(merged_data_date_time_feature, encode_columns)

bool_columns = ['subject_with_personalization', 'subject_with_deadline',
                'subject_with_emoji', 'subject_with_bonuses', 
                'subject_with_discount', 'subject_with_saleout']
merged_data_date_time_feature = process_bool_columns(merged_data_date_time_feature, bool_columns)

numerical_columns = ['duration', 'subject_length', 'sent_at_weekday', 'sent_at_hour', 
                     'sent_at_month', 'opened_last_time_at_weekday', 
                     'opened_last_time_at_hour', 'opened_last_time_at_month', 
                     'clicked_last_time_at_weekday', 'clicked_last_time_at_hour', 
                     'clicked_last_time_at_month']
merged_data_date_time_feature = convert_to_numeric(merged_data_date_time_feature, numerical_columns)

# Kiểm tra dữ liệu sau khi xử lý
merged_data_date_time_feature

,duration,subject_length,subject_with_personalization,subject_with_deadline,subject_with_emoji,subject_with_bonuses,subject_with_discount,subject_with_saleout,sent_at_weekday,sent_at_hour,sent_at_month,opened_last_time_at_weekday,opened_last_time_at_hour,opened_last_time_at_month,clicked_last_time_at_weekday,clicked_last_time_at_hour,clicked_last_time_at_month,topic_encoded,channel_x_encoded
0,22.0,133.0,0,0,1,0,0,0,2,7,5,3,2,5,3,2,5,2,0
1,31451.0,133.0,0,0,1,0,0,0,2,7,5,2,20,5,2,20,5,2,0
2,37.0,133.0,0,0,1,0,0,0,2,7,5,3,11,5,3,11,5,2,0
3,12.0,133.0,0,0,1,0,0,0,2,7,5,3,11,5,3,11,5,2,0
4,10.0,133.0,0,0,1,0,0,0,2,7,5,2,16,5,2,16,5,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86506,44.0,87.0,0,1,1,0,0,0,4,8,6,4,8,6,4,8,6,2,0
86507,1639.0,87.0,0,1,1,0,0,0,4,8,6,4,9,6,4,9,6,2,0
86508,7.0,87.0,0,1,1,0,0,0,4,8,6,4,8,6,4,8,6,2,0
86509,9.0,87.0,0,1,1,0,0,0,4,8,6,4,12,6,4,12,6,2,0


In [19]:
# Xóa các dòng bị trùng lặp
cleaned_data = merged_data_date_time_feature.drop_duplicates()
cleaned_data

,duration,subject_length,subject_with_personalization,subject_with_deadline,subject_with_emoji,subject_with_bonuses,subject_with_discount,subject_with_saleout,sent_at_weekday,sent_at_hour,sent_at_month,opened_last_time_at_weekday,opened_last_time_at_hour,opened_last_time_at_month,clicked_last_time_at_weekday,clicked_last_time_at_hour,clicked_last_time_at_month,topic_encoded,channel_x_encoded
0,22.0,133.0,0,0,1,0,0,0,2,7,5,3,2,5,3,2,5,2,0
1,31451.0,133.0,0,0,1,0,0,0,2,7,5,2,20,5,2,20,5,2,0
2,37.0,133.0,0,0,1,0,0,0,2,7,5,3,11,5,3,11,5,2,0
3,12.0,133.0,0,0,1,0,0,0,2,7,5,3,11,5,3,11,5,2,0
4,10.0,133.0,0,0,1,0,0,0,2,7,5,2,16,5,2,16,5,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86500,9.0,87.0,0,1,1,0,0,0,4,8,6,1,23,6,1,23,6,2,0
86502,31.0,87.0,0,1,1,0,0,0,4,8,6,5,0,6,5,0,6,2,0
86503,64.0,87.0,0,1,1,0,0,0,4,8,6,3,5,6,2,6,6,2,0
86507,1639.0,87.0,0,1,1,0,0,0,4,8,6,4,9,6,4,9,6,2,0


In [20]:
# Tạo X và y cho mô hình học máy
X = cleaned_data.drop(columns=['duration'])  # Dữ liệu đặc trưng
y = cleaned_data['duration']  # Biến mục tiêu

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# Định nghĩa pipeline Linear Regression với Scaling và Feature Selection
regression_pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Chuẩn hóa dữ liệu
    ('feature_selection', SelectFromModel(RandomForestRegressor(random_state=42))),  # Chọn đặc trưng
    ('regressor', LinearRegression())  # Hồi quy tuyến tính
])

# Tìm tham số tối ưu bằng GridSearchCV
param_grid = {
    'feature_selection__estimator__n_estimators': [50, 100, 200],  # Số cây trong RandomForest
    'regressor__fit_intercept': [True, False]  # Tùy chọn fit_intercept
}
grid_search = GridSearchCV(regression_pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Đánh giá mô hình trên tập kiểm tra
y_pred = grid_search.best_estimator_.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Sử dụng Cross-Validation để đánh giá toàn diện
kf = KFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(grid_search.best_estimator_, X_train, y_train, cv=kf, scoring='neg_mean_squared_error')

# In kết quả
print("Linear Regression Best Parameters:", grid_search.best_params_)
print("Test Mean Squared Error:", mse)
print("Test R^2 Score:", r2)
print("Cross-Validation MSE Mean:", -cv_scores.mean())
print("Cross-Validation MSE Std:", cv_scores.std())

# In các đặc trưng được chọn
selected_features = X_train.columns[grid_search.best_estimator_.named_steps['feature_selection'].get_support()]
print("Selected Features:", list(selected_features))

Linear Regression Best Parameters: {'feature_selection__estimator__n_estimators': 50, 'regressor__fit_intercept': True}
Test Mean Squared Error: 71606144429.98453
Test R^2 Score: 0.0964318779222808
Cross-Validation MSE Mean: 70661521917.4559
Cross-Validation MSE Std: 5101917569.515907
Selected Features: ['subject_length', 'sent_at_hour', 'opened_last_time_at_weekday', 'opened_last_time_at_hour', 'clicked_last_time_at_weekday', 'clicked_last_time_at_hour', 'clicked_last_time_at_month']


Nhận xét về kết quả mô hình:
1. Các tham số tốt nhất:
Mô hình chọn:
- Số cây trong RandomForest (n_estimators): 50.
- Fit intercept: True, tức là mô hình sẽ ước lượng giá trị intercept (giao điểm với trục y).
2. Đánh giá trên tập kiểm tra:
Mean Squared Error (MSE): 71.61 tỷ.
- Giá trị MSE khá lớn, cho thấy sai số trung bình bình phương giữa giá trị thực tế và giá trị dự đoán còn cao.
R² Score: 0.096.
- Điểm R² rất thấp (gần 0), chứng tỏ mô hình chỉ giải thích được khoảng 9.6% phương sai của dữ liệu. Điều này cho thấy Linear Regression có thể không phải là mô hình tốt nhất cho bài toán này.
3. Đánh giá qua Cross-Validation:
- Cross-Validation MSE Mean: ~70.66 tỷ.
Kết quả MSE trung bình qua Cross-Validation gần với MSE trên tập kiểm tra, chứng tỏ mô hình hoạt động nhất quán trên các tập dữ liệu khác nhau.
- Cross-Validation MSE Std: ~5.1 tỷ.
Độ lệch chuẩn của MSE qua các fold khá nhỏ, cho thấy mô hình ổn định.
4. Các đặc trưng được chọn (Selected Features):
Các đặc trưng được chọn lọc bởi RandomForest là:
- subject_length, sent_at_hour, opened_last_time_at_weekday, opened_last_time_at_hour, clicked_last_time_at_weekday, clicked_last_time_at_hour, clicked_last_time_at_month.
Đây là các đặc trưng liên quan đến thời gian và tương tác của người dùng với email, cho thấy tầm quan trọng của các yếu tố thời gian và hành vi người dùng trong dự đoán.

Nhận định:
1. Hiệu suất mô hình:
Kết quả với R² = 0.096 cho thấy Linear Regression không phù hợp với dữ liệu này. Dữ liệu có thể có mối quan hệ phi tuyến tính, cần cân nhắc các mô hình phức tạp hơn như Random Forest, Gradient Boosting, hoặc XGBoost.

2. Cần cải thiện đặc trưng:
Dữ liệu có thể chứa nhiều đặc trưng không tuyến tính hoặc có tương tác phức tạp, do đó cần khám phá thêm các đặc trưng phái sinh hoặc sử dụng các kỹ thuật xử lý đặc trưng nâng cao (Feature Engineering).

3. Khả năng chọn đặc trưng:
RandomForest đã chọn ra các đặc trưng có ý nghĩa nhất. Tuy nhiên, có thể cần thử nghiệm với các thuật toán chọn đặc trưng khác như Lasso Regression hoặc Mutual Information để cải thiện.

4. Hướng cải thiện:
- Thử nghiệm mô hình khác: Sử dụng các mô hình phi tuyến tính như Random Forest Regressor, Gradient Boosting Regressor, hoặc XGBoost.
- Tăng số lượng cây trong RandomForest: Hiện tại, n_estimators = 50 là giá trị khá thấp, có thể tăng lên để cải thiện khả năng chọn đặc trưng.
- Xem xét bổ sung đặc trưng: Các đặc trưng khác như tương tác giữa các đặc trưng hiện có hoặc thêm thông tin về nội dung email có thể mang lại hiệu quả cao hơn.